In [1]:
from future.builtins import next
import os
import csv
import re
import collections
import logging
import optparse
from numpy import nan
import dedupe
from unidecode import unidecode
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
import json
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType,FloatType

# ## Logging

# Dedupe uses Python logging to show or suppress verbose output. Added for convenience.
# To enable verbose logging, run `python examples/csv_example.py -v`


print("PySpark Application Started")

spark = SparkSession \
    .builder \
    .appName("dedupe") \
    .master("local[*]") \
    .getOrCreate()

print(spark.sparkContext.appName)

PySpark Application Started


22/07/28 18:07:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


dedupe


In [23]:
input_file = '/home/abhishekks446/data/BigBasket_Products.csv'
training_file = '/home/abhishekks446/pyspark/my_project_dir/Untitled Folder/settings/csv_example_training.json'
primart_key="unique_id"

df= spark.read.option("header","true").csv(input_file).withColumnRenamed("index","unique_id").limit(500)

In [27]:
df.printSchema()

root
 |-- unique_id: string (nullable = true)
 |-- product: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- sale_price: string (nullable = true)
 |-- market_price: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- description: string (nullable = true)



In [28]:
df.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("/home/abhishekks446/data/output")

In [25]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    import unidecode
    # column = column.decode("utf8")
    column = unidecode.unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column :
        column = None
    return column
def readData():
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """
    
    data_d = {}
    results = df.toJSON().map(lambda j: json.loads(j)).collect()
    for row in results:
        clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
        row_id = row[primart_key]
        data_d[row_id] = dict(clean_row)

    return data_d